In [2]:
%classpath add mvn com.cra.figaro figaro_2.11 5.0.0.0

In [3]:
import com.cra.figaro.language._
import com.cra.figaro.algorithm._
import com.cra.figaro.library.compound.If

import com.cra.figaro.language._
import com.cra.figaro.algorithm._
import com.cra.figaro.library.compound.If


### Computing ranges

This can be done if there isn't an infinite number of elements in the universe and the operation can be defined.

Most built-in Element classes have a finite number of possible values, except for atomic continuous ones.

In [3]:
val u = Universe.universe
val values = Values(u)

com.cra.figaro.algorithm.Values@7268a25e

In [4]:
val e1 = Flip(0.7)
val e2 = If(e1, Select(.2 -> 1, .8 -> 2), Select(.4 -> 2, .6 -> 3))

If(Flip(0.7), Select(0.2 -> 1, 0.8 -> 2), Select(0.4 -> 2, 0.6 -> 3))

In [5]:
values(e2) // <- range

Set(1, 2, 3)

## Variable elimination

In [6]:
val e1 = Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25 -> 0.7, 0.25 -> 0.9)
val e2 = Flip(e1)
val e3 = If(e2, Select(0.3 -> 1, 0.7 -> 2), Constant(2))

If(Flip(Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25 -> 0.7, 0.25 -> 0.9)), Select(0.3 -> 1, 0.7 -> 2), Constant(2))

In [7]:
e3.setCondition((i: Int) => i == 2) // observation?

In [8]:
val ve = factored.VariableElimination(e2) // just an object for the inference

com.cra.figaro.algorithm.factored.ProbQueryVariableElimination@473d4a6b

In [9]:
ve.start()

In [10]:
ve.distribution(e2) // probabilities of each value

[[(0.4878048780487805,false), (0.5121951219512194,true)]]

*probability* returns the probability that value satisfies a given predicate

In [11]:
ve.probability(e2, true) // shorthand

0.5121951219512194

In [12]:
ve.probability(e2, (b: Boolean) => b) // equivalent

0.5121951219512194

In [13]:
ve.kill()

We can get the expected value that the inference result will have on a given function.

In [18]:
// This is pretty cool
ve.expectation(e2, (b: Boolean) => if (b) 3.0; else 1.5)

2.2682926829268286

In [21]:
factored.VariableElimination.probability(e2, true) // for convenience

0.5121951219512194

### Belief Propagation

In [14]:
import com.cra.figaro.algorithm.factored.beliefpropagation._

import com.cra.figaro.algorithm.factored.beliefpropagation._


In [23]:
val bp = BeliefPropagation(100, e2)

com.cra.figaro.algorithm.factored.beliefpropagation.BeliefPropagation$$anon$1@26413437

This creates a BP object that will pass messages on a factor graph created from the universe containing e1, e2 and e3

In [24]:
bp.start()

In [26]:
bp.expectation(e2, (b: Boolean) => if (b) 3.0; else 1.5)

2.268292682926829

### Lazy Factored Inference

Useful for infinitely recursive models. Expand factor graph to a bounded depth and quantify the effect of the unexplored graph on the query. Can thus get lower and upper bounds on the probability

In [ ]:
// ?

### Importance sampling

Combination of importance and rejection sampling. Multiplies the weight of the sample by the value of the constraint when constraint is encountered. 

In [15]:
import com.cra.figaro.algorithm.sampling._

import com.cra.figaro.algorithm.sampling._


In [17]:
val e1 = Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25-> 0.7, 0.25 -> 0.9)
val e2 = Flip(e1)

Flip(Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25 -> 0.7, 0.25 -> 0.9))

In [21]:
val e3 = If(e2, Select(0.3 -> 1, 0.7 -> 2), Constant(2))
e3.setCondition((i: Int) => i == 2) // observe 2?

null

In [33]:
val imp = Importance(10000, e2)
// "one-time" algorithm - cannot be re-used after sampling
// "anytime" version exists that can be queried as it runs
// to run the anytime version, omit the number of samples argument

com.cra.figaro.algorithm.sampling.Importance$$anon$1@50999379

In [37]:
imp.start()
println(imp.distribution(e2))
imp.kill()

Stream((0.4900000000000182,false), ?)


**Anytime version**

In [38]:
val imp = Importance(e2)
imp.start()
Thread.sleep(1000)
imp.stop()
println(imp.probability(e2, (b: Boolean) => b))
imp.kill()

0.5139434533393535


null

## MCMC

In [4]:
val e1 = Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25 -> 0.7, 0.25 -> 0.9)
val e2 = Flip(e1)
val e3 = If(e2, Select(0.3 -> 1, 0.7 -> 2), Constant(2))
e3.setCondition((i: Int) => i == 2)
// same model as above

null

In [55]:
val mh = MetropolisHastings(1000, ProposalScheme.default, e2)

com.cra.figaro.algorithm.sampling.OneTimeMetropolisHastings@1fc66abd

In [56]:
mh.start()
println(mh.distribution(e2))
mh.kill()

Stream((0.589,false), ?)


### Gibbs sampling

Gibbs sampling iterates through each variable in the model and samples them conditioned on the rest of the model (actually the Markov blanket, as it's all that's needed). 

In [5]:
import com.cra.figaro.algorithm.factored.gibbs._

import com.cra.figaro.algorithm.factored.gibbs._


In [6]:
val e1 = Select(0.25 -> 0.3, 0.25 -> 0.5, 0.25 -> 0.7, 0.25 -> 0.9)
val e2 = Flip(e1)
val e3 = If(e2, Select(0.3 -> 1, 0.7 -> 2), Constant(2))
e3.setCondition((i: Int) => i == 2)
// same model as above

null

In [10]:
val gs = Gibbs(500, e2)

com.cra.figaro.algorithm.factored.gibbs.Gibbs$$anon$1@54192bf5

In [12]:
gs.start()
println(gs.distribution(e3))
gs.kill()

com.cra.figaro.algorithm.AlgorithmActiveException: com.cra.figaro.algorithm.AlgorithmActiveException